In [40]:
import time
import numpy as np
from keras.utils import np_utils
import keras.callbacks as cb
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
import feather 
from mailerWithUtf8 import mail
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json

# basic config

In [41]:
classify_num = 12
epochs = 300
batch = 128

# preprocess load data

In [42]:
start_time = time.time()
path = 'C:/Users/VIPLAB/Desktop/preprocess_py/marketing_analyze/april_marketing_anylze_numeric_and_bool_group11_max_min_train.feather' 
train_df = feather.read_dataframe(path)
print('load feather %s seconds', format(time.time() - start_time))


load feather %s seconds 0.8458900451660156


In [43]:
start_time = time.time()
path = 'C:/Users/VIPLAB/Desktop/preprocess_py/marketing_analyze/april_marketing_anylze_numeric_and_bool_group11_max_min_test.feather' 
test_df = feather.read_dataframe(path)
print('load feather %s seconds', format(time.time() - start_time))

load feather %s seconds 0.4078183174133301


In [44]:
train_x, train_y = train_df.iloc[:, 0:-1].values, train_df.iloc[:, -1].values

In [45]:
test_x, test_y = test_df.iloc[:, 0:-1].values, test_df.iloc[:, -1].values

In [46]:
train_x = train_x.reshape(train_x.shape[0], -1)   # normalize
test_x = test_x.reshape(test_x.shape[0], -1)   # normalize



In [47]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_y)
encoded_Y = encoder.transform(train_y)
# convert integers to dummy variables (i.e. one hot encoded)
train_y = np_utils.to_categorical(encoded_Y)

encoded_Y = encoder.transform(test_y)
# convert integers to dummy variables (i.e. one hot encoded)
test_y = np_utils.to_categorical(encoded_Y)

In [48]:
len(train_y)

1569788

In [49]:
len(test_y)

773180

# construct and compile model

In [50]:
start_time = time.time()
print ('Compiling Model ... ')
model = Sequential()
model.add(Dense(100, input_shape=train_x[0].shape ))
model.add(Activation('relu'))
# model.add(Dropout(0.2))
model.add(Dense(70))
model.add(Activation('relu'))
model.add(Dense(30))
model.add(Activation('relu'))
# model.add(Dropout(0.2))
model.add(Dense(12))
model.add(Activation('softmax'))

rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=rms,
          metrics=['accuracy'])
print('Model compield in {0} seconds'.format(time.time() - start_time))


Compiling Model ... 
Model compield in 0.08525252342224121 seconds


# Train model

In [ ]:
start_time = time.time()
print('Training model...')
model.fit(train_x, train_y , epochs=epochs, batch_size=batch,
          shuffle=True,validation_split=0.1, verbose=2)
print("Training duration : {0}".format(time.time() - start_time))


Training model...
Train on 1412809 samples, validate on 156979 samples
Epoch 1/300
43s - loss: 1.7780 - acc: 0.3814 - val_loss: 1.7478 - val_acc: 0.3872
Epoch 2/300
46s - loss: 1.7461 - acc: 0.3924 - val_loss: 1.7405 - val_acc: 0.3955
Epoch 3/300
48s - loss: 1.7406 - acc: 0.3952 - val_loss: 1.7393 - val_acc: 0.3954
Epoch 4/300
45s - loss: 1.7384 - acc: 0.3974 - val_loss: 1.7415 - val_acc: 0.3967
Epoch 5/300
46s - loss: 1.7361 - acc: 0.3990 - val_loss: 1.7336 - val_acc: 0.4015
Epoch 6/300
46s - loss: 1.7325 - acc: 0.4003 - val_loss: 1.7390 - val_acc: 0.3981
Epoch 7/300
47s - loss: 1.7315 - acc: 0.4015 - val_loss: 1.7195 - val_acc: 0.4032
Epoch 8/300
45s - loss: 1.7318 - acc: 0.4021 - val_loss: 1.7697 - val_acc: 0.3899
Epoch 9/300
44s - loss: 1.7324 - acc: 0.4030 - val_loss: 1.7217 - val_acc: 0.4049
Epoch 10/300
46s - loss: 1.7299 - acc: 0.4038 - val_loss: 1.7293 - val_acc: 0.4047
Epoch 11/300
45s - loss: 1.7306 - acc: 0.4040 - val_loss: 1.7206 - val_acc: 0.4048
Epoch 12/300
46s - loss: 

In [ ]:
train_time = time.time() - start_time

# save model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 


# test model and get accuracy

In [ ]:
test_batch = 32
loss, accuracy = model.evaluate(test_x, test_y, batch_size=test_batch)

In [ ]:
print("%s: %.2f%%" % (model.metrics_names[1], accuracy*100))


# send mail to alert

In [ ]:
model_config = []
for index, layer in enumerate(model.get_config()):
    model_config.append("layer - > %d===============" % index)
    model_config.append("class_name = " + layer["class_name"] )
    if(str(layer["config"].get("units", "None")) != "None"):
        model_config.append("units = " + str(layer["config"].get("units", "None")))
    if(str(layer["config"].get("activation", "None")) != "None"):
        model_config.append("activation = " + layer["config"]["activation"])
    if(str(layer["config"].get("rate", "None")) != "None"):
        model_config.append("rate = " + str(layer["config"]["rate"]))
model_info = '\n'.join(model_config)

In [ ]:
model_info += "\n\ntrain epochs = " + str(epochs)
model_info += "\ntrain batch = " + str(batch)
model_info += "\ntest batch = " + str(test_batch)
model_info += "\ntrain time = " + str(train_time)
model_info += "\nloss = " + str(loss)
model_info += "\n\n\naccuracy = " + str(accuracy)



In [ ]:
test=mail()
test.main("model finished", model_info)

# write readme to report the model

In [ ]:
with open('readme.md', 'w+') as f:
     f.write(model_info)
f.closed

In [110]:
# for i in range(10):
#     probabilities = model.predict(test_x[i:i+1,:], batch_size=32, verbose=0)
#     probabilities = model.predict(test_x[i:i+1,:])
#     probabilities = model.predict(test_x)
#     probabilities[probabilities>=0.5] = 1
#     probabilities[probabilities<0.5] = 0
#     print(probabilities)

[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
